In [1]:
import sys
import os
from dotenv import load_dotenv

load_dotenv()

# 取得專案根目錄 (lib 的父目錄)
root_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

# 將 lib 加入 Python 模組搜尋路徑
sys.path.append(os.path.join(root_dir, "lib"))

In [2]:
from agent import Agent
from tools import search
import re
import logging

react_system_prompt = """You are an AI Agent based on the ReAct framework, tasked with answering questions through the Observe -> Reason -> Act loop.

## **Action Rules**
1. **Reason**  
   - First, output your thinking process based on the current information, using natural language to explain your reasoning.
   
2. **Act**  
   - Execute available tools using **Python syntax**:
   - Actions you can use:
     ```python
     search(query: str) -> List[Dict]  # Get results through a search engine
     final_answer() -> None  # Provide the final answer directly
     ```
   - Your actions must be written in Python code blocks:
     ```python
     search("What is dark energy?")
     ```

## **Observation Rules**
- If you receive `<retrieved data>`, it contains search results that you should use for your reasoning.

## **Examples**

### **Searching for Information**
'''
...Based on the question, I should first search for "Who discovered dark energy?" to get background knowledge.
```python
search("Who discovered dark energy?")
```
'''

### **Answering After Obtaining Sufficient Information**
'''
...Based on the retrieved data, I now have enough information to answer the question.

```python
final_answer()
```
'''
"""

In [ ]:
class ReAct:
    def __init__(self, llm, system_prompt, max_turns=10, log_file="react_debug.log"):
        self.agent = Agent(llm=llm)
        self.max_turns = max_turns
        self.history = [{"role": "developer", "content": system_prompt}]

        # Setup logging
        logging.basicConfig(filename=log_file, level=logging.DEBUG,
                            format="%(asctime)s [%(levelname)s] %(message)s", encoding="utf-8")
        logging.info("\n=== New ReAct Execution Started ===\n")

    def parse_and_execute(self, llm_output: str):
        """
        Parses the LLM's output and executes the corresponding Python action safely.
        """
        logging.info(f"LLM Output:\n{llm_output}")

        match = re.search(r"```python\n(.*?)\n```", llm_output, re.DOTALL)
        if match:
            code = match.group(1).strip()
            logging.info(f"Parsed Action:\n{code}")

            # Match search() or final_answer()
            search_match = re.fullmatch(r'search\("([^"]+)"\)', code)
            final_match = re.fullmatch(r'final_answer\(\)', code)  # Ensure no arguments

            if search_match:
                query = search_match.group(1).strip()
                logging.info(f"Executing search for: {query}")
                return "search", search(query, max_results=5)
            elif final_match:
                logging.info("Triggering final_answer()")
                return "answer", self.final_answer()
            else:
                logging.warning("Invalid action format detected. Retrying...")
                return "retry", None
        else:
            logging.warning(
                "No valid Python action found in LLM output. Retrying...")
            return "retry", None

    def final_answer(self):
        """
        Generates a final answer based on the entire conversation history.
        """
        logging.info("Generating final answer...")

        conversations = [
            {"role": "developer", "content": "You are an AI assistant designed to answer human questions. Your task is to strictly adhere to the conversation context and integrate information to respond to inquiries."}]
        conversations += self.history[1:]
        conversations += [{"role": "user", "content": "Please respond to the question based on the conversation content above."}]

        response = self.agent.generate_response(conversations)
        logging.info(f"Final Answer: {response}")
        return response

    def run(self, question):
        """
        執行 ReAct 迴圈：
        - 產生 Thought, Action
        - 執行 Action 取得 Observation
        - 持續迴圈直到獲得答案
        """
        logging.info(f"Starting new session with question: {question}")

        self.history.append(
            {"role": "user", "content": f"Question: {question}"})
        conversations = self.history.copy()
        turn = 1
        while turn <= self.max_turns:
            logging.info(f"Turn {turn}: Generating Thought & Action...")

            # 1. Generate Thought + Action
            response = self.agent.generate_response(conversations)
            logging.info(f"LLM Response:\n{response}")

            # 2. Execute Action
            state, feedback = self.parse_and_execute(response)

            # 3. Retry if LLM output is invalid
            if state == "retry":
                logging.warning(f"Retrying Turn {turn}...")
                continue  # Do not increment turn, just retry the step

            # 4. Update Conversations
            if state == "search":
                # Append new LLM response and retrieval data
                self.history.append({"role": "assistant", "content": response})
                search_observation = {
                    "role": "user", "content": f"<retrieved data>\n{feedback}\n</retrieved data>"}

                conversations = self.history.copy() + [search_observation]

                logging.info(f"Search Results for Turn {turn}: {feedback}\n")

            elif state == "answer":
                logging.info("Final Answer Reached.")
                return feedback

            else:
                logging.error("Unexpected action detected. Skipping.")

            turn += 1

        logging.warning("Max turns reached. No definitive answer found.")
        return self.final_answer()

In [4]:
react_agent = ReAct(llm="gpt-4o-mini",
                    system_prompt=react_system_prompt, max_turns=10)
react_agent.run(question="are building societies safer than banks?")

'建築社（building societies）和銀行（banks）都是金融機構，但在安全性方面存在一些差異。通常，兩者都受到金融監管機構的監管，以確保存款人的資金安全。以下是一些比較的要點：\n\n1. **結構和目的**：建築社通常是為了服務會員而設立的，主要提供房貸和儲蓄產品，而銀行則是為了盈利，提供更廣泛的金融服務。\n\n2. **存款保護**：在許多國家，銀行和建築社都參加了存款保護計劃，例如在英國，建築社和銀行的存款均受金融服務補償計劃（FSCS）的保障，最多可保護每位客戶至£85,000。\n\n3. **風險管理**：銀行可能會參與更多高風險的投資業務，而建築社的業務模式通常較為保守，主要集中於住宅貸款和儲蓄。\n\n4. **資本充足率**：建築社和銀行都有資本充足率的要求，但具體的規定可能會有所不同，影響其財務穩定性。\n\n總體來說，哪一種機構更安全取決於多種因素，包括監管環境、管理實踐及個別機構的財務健康狀況。在選擇存款機構時，最好考慮其公共評級、財務報告及過去的業務表現。'

In [ ]:
import re
from tools import search  # Import the search function

def parse_and_execute(llm_output: str):
    """
    Parses the LLM's output and executes the corresponding Python action safely.
    """
    match = re.search(r"```python\n(.*?)\n```", llm_output, re.DOTALL)
    if match:
        code = match.group(1).strip()  # Extract the Python code
        print(f"🔍 Parsed Action:\n{code}")

        # Match the function call format: `search("query")` or `final_answer("answer")`
        search_match = re.match(r'search\("(.+)"\)', code)
        final_match = re.match(r'final_answer\("(.+)"\)', code)

        if search_match:
            query = search_match.group(1)
            return search(query, max_results=10)  # Call the actual search function
        elif final_match:
            return final_match.group(1)  # Just return the final answer
        else:
            print("⚠️ Invalid action format detected.")
            return None
    else:
        print("⚠️ No valid Python action found in LLM output.")
        return None

test = '''
根據問題，我應該先搜尋 "Who discovered dark energy?" 來獲取背景知識。
```python
search("Who discovered dark energy?")
```
'''
parse_and_execute(test)
